# ***Model Traning And Pipeline Creation***

## ***Building pipeline With TfIDF Vectorization***

#### ***Importing Library***

In [4]:
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import joblib
import emoji
import re

from sklearn import set_config
set_config(display='diagram')


ModuleNotFoundError: No module named 'emoji'

In [ ]:
# Ensure NLTK downloads are available
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

#### **Load DataSet**

In [ ]:
ds = pd.read_csv("C:\\Users\\ujjwa_n18433z\\Desktop\\ujjwal\\All Projects\\Project_WoC_7.0_Fake_Review_Detection\\fakeReviewData.csv")


#### **Text Preprocessor Class**


In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stopwords = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()


    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        # Convert emojis to text
        text = emoji.demojize(text, delimiters=("", ""))

        # Handle contractions
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"'m", " am", text)
        text = re.sub(r"'s", " is", text)
        text = re.sub(r"'re", " are", text)
        text = re.sub(r"'ve", " have", text)
        text = re.sub(r"'d", " would", text)
        text = re.sub(r"'ll", " will", text)

        # Remove URLs and email addresses
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        text = re.sub(r"\S*@\S*\s?", "", text)

        # Remove special characters
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

        # Remove digits and lowercase
        text = "".join([char.lower() for char in text if not char.isdigit()])

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize
        words = word_tokenize(text)

        # Remove stopwords
        words = [word for word in words if word not in self.stopwords]

        # Lemmatize
        words = [self.lemmatizer.lemmatize(word) for word in words]  # Lemmatization

        # Remove multiple spaces
        text = re.sub(r"\s+", " ", " ".join(words)).strip()

        # Rejoin words into a single string
        return text

#### **Encoding Labels**

In [ ]:
target_encoder = LabelEncoder()
ds['label'] = target_encoder.fit_transform(ds['label'])  # Encode target (e.g., 'CG' -> 0, 'OR' -> 1)

In [ ]:
# Get the mapping
dict(zip(target_encoder.classes_, range(len(target_encoder.classes_))))

#### **Creating a Column Transformation**

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text', Pipeline([
            ('preprocessor', TextPreprocessor()),
            ('tfidf', TfidfVectorizer(max_features=500))
        ]), 'text_'),
        ('category', OneHotEncoder(handle_unknown='ignore'), ['category']),
        ('rating', StandardScaler(), ['rating'])  # Wrap 'rating' in a list to make it 2D
    ],
    remainder='passthrough'
)

#### **Split the data into training and testing sets**

In [ ]:
# Split the data into training and testing sets
X = ds.drop(columns=['label']) 
y = ds['label'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


#### **Define Full Pipeline**

In [ ]:
pipelinelg = Pipeline([
    ('preprocessor', preprocessor),                    
    ('classifier', LogisticRegression(random_state=42))  # Logistic Regression
])

In [ ]:
pipelinelg

#### **Train, predict and evaluate Logistic Regression**

In [ ]:
# Train the pipeline
pipelinelg.fit(X_train, y_train)

# Generate predictions
y_pred = pipelinelg.predict(X_test)

# Create Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipelinelg.named_steps['classifier'].classes_)

# Plot the Confusion Matrix
cmd.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))


#### **Define Support Vector Classifier Pipeline**

In [ ]:


pipelinesvm = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42))
])

#### **Train, predict and evaluate SVM**

In [ ]:
# Train the pipeline
pipelinesvm.fit(X_train, y_train)

# Generate predictions
y_pred = pipelinesvm.predict(X_test)

# Create Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipelinesvm.named_steps['classifier'].classes_)

# Plot the Confusion Matrix
cmd.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

#### **Define Random Forest Classifier Pipeline**

In [ ]:

pipelinerf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [ ]:
pipelinerf

#### **Train, predict and evaluate Random Forest**

In [ ]:
pipelinerf.fit(X_train, y_train)
 
# Generate predictions
y_pred = pipelinerf.predict(X_test)

# Create Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipelinerf.named_steps['classifier'].classes_)

# Plot the Confusion Matrix
cmd.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

#### **Define Multi Layer Perceptron Classifier Pipeline**

In [ ]:

pipelinemlp = Pipeline([
    ('preprocessor', preprocessor),  
    ('classifier', MLPClassifier(
        hidden_layer_sizes=(100, 50),
        activation='relu',           
        solver='adam',               
        max_iter=200,                
        random_state=42,
        early_stopping=True,         
        validation_fraction=0.1      
    ))
])

In [ ]:
pipelinemlp

#### **Train, predict and evaluate MLP**

In [ ]:
# Train the pipeline
pipelinemlp.fit(X_train, y_train)
 
# Generate predictions
y_pred = pipelinemlp.predict(X_test)

# Create Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipelinemlp.named_steps['classifier'].classes_)

# Plot the Confusion Matrix
cmd.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

## ***Building Pipeline With Embedding*** 

#### **Word2Vec Model Preparation**

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stopwords = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()


    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        # Convert emojis to text
        text = emoji.demojize(text, delimiters=("", ""))

        # Handle contractions
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"'m", " am", text)
        text = re.sub(r"'s", " is", text)
        text = re.sub(r"'re", " are", text)
        text = re.sub(r"'ve", " have", text)
        text = re.sub(r"'d", " would", text)
        text = re.sub(r"'ll", " will", text)

        # Remove URLs and email addresses
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        text = re.sub(r"\S*@\S*\s?", "", text)

        # Remove special characters
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

        # Remove digits and lowercase
        text = "".join([char.lower() for char in text if not char.isdigit()])

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize
        words = word_tokenize(text)

        # Lemmatize
        words = [self.lemmatizer.lemmatize(word) for word in words]  # Lemmatization

        # Remove multiple spaces
        text = re.sub(r"\s+", " ", " ".join(words)).strip()

        # Rejoin words into a single string
        return text

In [ ]:
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model=None, vector_size=100):
        self.model = model
        self.vector_size = vector_size
    
    def fit(self, X, y=None):
        if self.model is None:
            sentences = [text.split() for text in X]
            self.model = Word2Vec(sentences, vector_size=self.vector_size, window=5, min_count=1, workers=4)
        return self
    
    def transform(self, X):
        vectors = []
        for text in X:
            words = text.split()
            word_vectors = [self.model.wv[word] for word in words if word in self.model.wv]
            if word_vectors:
                vectors.append(np.mean(word_vectors, axis=0))
            else:
                vectors.append(np.zeros(self.vector_size))  # Return a zero vector if no words are found
        return np.array(vectors, dtype=np.float32)  # Ensure output is of type float32

#### **Define Column Transformer with Word2Vec**

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text', Pipeline([
            ('preprocessor', TextPreprocessor()),
            ('word2vec', Word2VecTransformer(vector_size=300))
        ]), 'text_'),
        ('category', OneHotEncoder(handle_unknown='ignore'), ['category']),
        ('rating', StandardScaler(), ['rating'])  
    ],
    remainder='passthrough'
)


#### **Define MLP Pipeline with Word2Vec Preprocessing**

In [ ]:

pipelinemlp_word2vec = Pipeline([
    ('preprocessor', preprocessor),  # Your ColumnTransformer
    ('classifier', MLPClassifier(
        hidden_layer_sizes=(100,50),  # Example: 2 hidden layers, 100 and 50 neurons
        activation='relu',              # ReLU activation
        solver='adam',                 # Adam optimizer
        max_iter=200,                  # Increase if needed for convergence
        random_state=42,
        early_stopping=True,          # Use early stopping to prevent overfitting
        validation_fraction=0.1       # Proportion of training data for validation
    ))
])

In [ ]:
pipelinemlp_word2vec


#### **Train, predict and evaluate MLP with Word2Vec**

In [ ]:
# Train the pipeline
pipelinemlp_word2vec.fit(X_train, y_train)
 
# Generate predictions
y_pred = pipelinemlp_word2vec.predict(X_test)

# Create Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.array(['CG', 'OR']))

# Plot the Confusion Matrix
cmd.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

#### **Saving Models**

In [ ]:
joblib.dump(pipelinemlp, 'best_mlp_word2vec_model.pkl')

## ***Building Pipeline With Spelling Correction and Embedding*** 

#### **Spelling Correction Class**

In [ ]:
# --- Spelling Correction Transformer ---
class SpellingCorrectionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lang='en'): # add language
        self.spell = Speller(lang=lang) # initialize Speller

    def fit(self, X, y=None):
        return self  # Nothing to fit, so just return self

    def transform(self, X):
        corrected_texts = []
        for text in X:
            # 1. Correct Spelling
            corrected_words = [self.spell(word) for word in text.split()] 
            corrected_text = " ".join(corrected_words)  

            corrected_texts.append(corrected_text)  
        return corrected_texts

#### **Define Column Transformer with Spelling Correction ,Text Processing ,Word2Vec**

In [ ]:
# --- ColumnTransformer ---
text_column = 'text_'
preprocessor = ColumnTransformer(
    transformers=[
        ('text', Pipeline([
            ('spelling', SpellingCorrectionTransformer()),
            ('preprocessor', TextPreprocessor()),
            ('word2vec', Word2VecTransformer(vector_size=150))
        ]), text_column),
        ('category', OneHotEncoder(handle_unknown='ignore',drop="first"), ['category']),
        ('rating', StandardScaler(), ['rating'])
    ],
    remainder='passthrough'  # or 'drop' if you don't want to keep remaining columns
)

#### **Define MLP Pipeline with Spelling Correction ,Text Processing ,Word2Vec Preprocessing**

In [ ]:
pipelinemlp_sc_word2vec = Pipeline([
    ('preprocessor', preprocessor),
    ('model',MLPClassifier(
    hidden_layer_sizes=(150, 70,30),  
    activation='relu',              
    solver='adam',                
    max_iter=200,                  
    random_state=42,
    early_stopping=True,          
    validation_fraction=0.1       
))  
])

#### **Train, predict and evaluate MLP with Spelling Correctin and Word2Vec**

In [ ]:

# --- Train and Evaluate MLPClassifier ---
pipelinemlp_sc_word2vec.fit(X_train, y_train)
y_pred = pipelinemlp_sc_word2vec.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.array(['CG', 'OR']))
cmd.plot(cmap=plt.cm.Blues)
plt.title("MLP Confusion Matrix")
plt.show()

print("MLP Classification Report:")
print(classification_report(y_test, y_pred))

## ***Building Pipeline With Pos Tagging and Embedding***

In [ ]:

# --- POS Tagger Transformer ---
class PosTaggerTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        pos_tags = []
        for text in X:
            tokens = nltk.word_tokenize(text)
            tags = nltk.pos_tag(tokens)  
            pos_tags.append([tag for word, tag in tags])
        return pos_tags
    

# --- POS Encoder Transformer ---
class PosEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')
        self.is_fitted = False

    def fit(self, X, y=None):
        all_pos_tags = [tag for sublist in X for tag in sublist] 
        self.ohe.fit(np.array(all_pos_tags).reshape(-1, 1)) 
        self.is_fitted = True
        return self

    def transform(self, X):
        if not self.is_fitted:
            raise ValueError("PosEncoderTransformer must be fitted before transform.")
        encoded_pos = []
        for doc_pos_tags in X:
            encoded_tags_for_doc = self.ohe.transform(np.array(doc_pos_tags).reshape(-1, 1))
            encoded_pos.append(encoded_tags_for_doc.mean(axis=0)) #average the encodings
        return np.array(encoded_pos)

In [ ]:
class ColumnCreator(BaseEstimator, TransformerMixin):
    def __init__(self, new_column_name, source_column):  
        self.new_column_name = new_column_name
        self.value = source_column
    def fit(self, X, y=None):
        return self

    def transform(self, X):

        # Apply a constant value
        X[self.new_column_name] = self.value
        return X


In [ ]:
# --- ColumnTransformer ---
text_column = 'text_'
preprocessor = ColumnTransformer(
    transformers=[
        ('word2vec', Pipeline([
            ('preprocessor', TextPreprocessor()),
            ('word2vec', Word2VecTransformer(vector_size=150)) 
        ]), text_column),

        ('pos', Pipeline([
            ('preprocessor', TextPreprocessor()),  
            ('pos_tagger', PosTaggerTransformer()), 
            ('pos_encoder', PosEncoderTransformer())
        ]), 'text_pos'),

        ('category', OneHotEncoder(handle_unknown='ignore', drop="first"), ['category']),
        ('rating', StandardScaler(), ['rating'])
    ],
    remainder='passthrough'  
)

In [ ]:
mlp_classifier = Pipeline([
    ('create_text_pos', ColumnCreator(new_column_name='text_pos',source_column='text_')), 
    ('preprocessor', preprocessor),
    ('model', MLPClassifier(
        hidden_layer_sizes=(150, 70, 30),
        activation='relu',
        solver='adam',
        max_iter=200,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    ))
])

In [ ]:
# --- Train and Evaluate MLPClassifier ---

mlp_classifier.fit(X_train, y_train)
y_pred = mlp_classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
cmd.plot(cmap=plt.cm.Blues)
plt.title("MLP Confusion Matrix")
plt.show()

print("MLP Classification Report:")
print(classification_report(y_test, y_pred))